# RAG Doc Agent (MVP)

Цель: вопрос → top-k релевантных фрагментов (evidence: источник/страница/url) → (опционально) LLM-ответ по контексту.

Pipeline: индексирование (чанки+эмбеддинги+векторный индекс) → извлечение top-k → (опц.) генерация ответа. 


## Импорт библиотек и загрузка данных

In [1]:
%pip install -q sentence-transformers langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import json
from typing import List, Dict, Any, Tuple

import numpy as np
import pandas as pd
from tqdm import tqdm

import faiss
import torch

from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))

FAISS_HAS_GPU = hasattr(faiss, "StandardGpuResources")
print("faiss has gpu support:", FAISS_HAS_GPU)


D:\Conda\envs\nlp-gpu\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


torch: 2.5.1
cuda available: True
gpu: NVIDIA GeForce RTX 4050 Laptop GPU
faiss has gpu support: True


In [3]:
# import os, sys, importlib.util, glob

# print("CWD:", os.getcwd())
# print("sys.path[0:5]:", sys.path[:5])

# spec = importlib.util.find_spec("torch")
# print("torch spec:", spec)
# print("torch origin:", getattr(spec, "origin", None))
# print("torch locations:", getattr(spec, "submodule_search_locations", None))

# print("\nPossible local conflicts:")
# print("torch.py:", glob.glob("torch.py"))
# print("torch/*:", glob.glob("torch/*")[:10])
# print("__pycache__ torch:", glob.glob("__pycache__/torch*")[:10])


In [4]:
PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  
RAW_DIR = os.path.join(PROJECT_DIR, "data", "raw")
INDEX_DIR = os.path.join(PROJECT_DIR, "data", "index")
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(INDEX_DIR, exist_ok=True)

WEBSITES_PATH_DEFAULT = "datasets/websites.csv"
QUESTIONS_PATH_DEFAULT = "datasets/questions_clean.csv"
SAMPLE_SUB_PATH_DEFAULT = "datasets/sample_submission_.csv"

WEBSITES_PATH = os.getenv("WEBSITES_PATH", WEBSITES_PATH_DEFAULT)
QUESTIONS_PATH = os.getenv("QUESTIONS_PATH", QUESTIONS_PATH_DEFAULT)
SAMPLE_SUB_PATH = os.getenv("SAMPLE_SUB_PATH", SAMPLE_SUB_PATH_DEFAULT)

if not os.path.exists(WEBSITES_PATH):
    alt = os.path.join(RAW_DIR, "websites.csv")
    if os.path.exists(alt): WEBSITES_PATH = alt

if not os.path.exists(QUESTIONS_PATH):
    alt = os.path.join(RAW_DIR, "questions_clean.csv")
    if os.path.exists(alt): QUESTIONS_PATH = alt

if not os.path.exists(SAMPLE_SUB_PATH):
    alt = os.path.join(RAW_DIR, "sample_submission.csv")
    if os.path.exists(alt): SAMPLE_SUB_PATH = alt

print("WEBSITES_PATH:", WEBSITES_PATH, os.path.exists(WEBSITES_PATH))
print("QUESTIONS_PATH:", QUESTIONS_PATH, os.path.exists(QUESTIONS_PATH))
print("SAMPLE_SUB_PATH:", SAMPLE_SUB_PATH, os.path.exists(SAMPLE_SUB_PATH))


WEBSITES_PATH: datasets/websites.csv True
QUESTIONS_PATH: datasets/questions_clean.csv True
SAMPLE_SUB_PATH: datasets/sample_submission_.csv True


In [5]:
web_df = pd.read_csv(WEBSITES_PATH)
q_df = pd.read_csv(QUESTIONS_PATH)
sub_df = pd.read_csv(SAMPLE_SUB_PATH)

display(web_df.head(2))
display(q_df.head(2))
display(sub_df.head(2))

print("websites:", web_df.shape, "questions:", q_df.shape)
print("web columns:", list(web_df.columns))
print("q columns:", list(q_df.columns))


,web_id,url,kind,title,text
0,1,https://alfabank.ru/,html,"Альфа-Банк - кредитные и дебетовые карты, кред...",Рассчитайте выгоду\nРасчёт калькулятора предва...
1,2,https://alfabank.ru/a-club/,html,А-Клуб. Деньги имеют значение,Брокерские услуги\nОткрытие брокерского счёта ...


,q_id,query
0,1,Номер счета
1,2,Где узнать бик и счёт


,q_id,web_list
0,1,"[935, 687, 963, 1893, 1885]"
1,2,"[1660, 1882, 1040, 1090, 64]"


websites: (1937, 5) questions: (6977, 2)
web columns: ['web_id', 'url', 'kind', 'title', 'text']
q columns: ['q_id', 'query']


## Подготовка документов к ретривалу

In [6]:
def clean_text(t: str) -> str:
    if t is None:
        return ""
    t = str(t)
    t = t.replace("\u00a0", " ")
    t = re.sub(r"\s+", " ", t).strip()
    return t

# заполним пустые
for col in ["title", "text", "url", "kind"]:
    if col in web_df.columns:
        web_df[col] = web_df[col].fillna("").map(clean_text)

web_df["web_id"] = web_df["web_id"].astype(int)

# Документ = title + text (так обычно лучше для поиска)
web_df["doc_text"] = (web_df["title"] + "\n\n" + web_df["text"]).map(clean_text)

# фильтр совсем пустых
web_df = web_df[web_df["doc_text"].str.len() > 0].reset_index(drop=True)

print("after cleaning:", web_df.shape)


after cleaning: (1937, 6)


In [7]:
CHUNK_SIZE = int(os.getenv("CHUNK_SIZE", "900"))
CHUNK_OVERLAP = int(os.getenv("CHUNK_OVERLAP", "150"))

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", " ", ""],
)

chunks: List[Dict[str, Any]] = []
for row in tqdm(web_df.itertuples(index=False), total=len(web_df)):
    parts = splitter.split_text(row.doc_text)
    for j, part in enumerate(parts):
        chunks.append({
            "chunk_id": len(chunks),
            "web_id": int(row.web_id),
            "url": row.url,
            "title": row.title,
            "kind": row.kind if "kind" in web_df.columns else "",
            "chunk_no": j,
            "text": part
        })

chunks_df = pd.DataFrame(chunks)
print("chunks:", chunks_df.shape)
display(chunks_df.head(3))


100%|████████████████████████████████████████████████████████████████████████████| 1937/1937 [00:00<00:00, 2378.73it/s]

chunks: (18965, 7)


,chunk_id,web_id,url,title,kind,chunk_no,text
0,0,1,https://alfabank.ru/,"Альфа-Банк - кредитные и дебетовые карты, кред...",html,0,"Альфа-Банк - кредитные и дебетовые карты, кред..."
1,1,2,https://alfabank.ru/a-club/,А-Клуб. Деньги имеют значение,html,0,А-Клуб. Деньги имеют значение Брокерские услуг...
2,2,2,https://alfabank.ru/a-club/,А-Клуб. Деньги имеют значение,html,1,. Структурирование передачи капитала Альфа‑Кап...


In [8]:
EMBED_MODEL = os.getenv("EMBED_MODEL", "intfloat/multilingual-e5-small")

device = "cuda" if torch.cuda.is_available() else "cpu"
embedder = SentenceTransformer(EMBED_MODEL, device=device)
print("EMBED_MODEL:", EMBED_MODEL, "| device:", device)

def format_e5(texts: List[str], is_query: bool) -> List[str]:
    name = (EMBED_MODEL or "").lower()
    if "e5" in name:
        prefix = "query: " if is_query else "passage: "
        return [prefix + t for t in texts]
    return texts

@torch.inference_mode()
def embed_texts(texts: List[str], is_query: bool, batch_size: int = 64) -> np.ndarray:
    texts = format_e5(texts, is_query=is_query)
    vecs = embedder.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=False,
        convert_to_numpy=True,
        normalize_embeddings=False
    ).astype("float32")
    return vecs


EMBED_MODEL: intfloat/multilingual-e5-small | device: cuda


In [9]:
texts = chunks_df["text"].tolist()
X = embed_texts(texts, is_query=False, batch_size=64)
print("X:", X.shape, X.dtype)


X: (18965, 384) float32


In [10]:
faiss.normalize_L2(X)
dim = X.shape[1]

cpu_index = faiss.IndexFlatIP(dim)
cpu_index.add(X)
print("cpu index ntotal:", cpu_index.ntotal)

USE_FAISS_GPU = (os.getenv("USE_FAISS_GPU", "1") == "1") and FAISS_HAS_GPU and torch.cuda.is_available()

index = cpu_index
gpu_res = None
if USE_FAISS_GPU:
    gpu_res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(gpu_res, 0, cpu_index)
    print("Using FAISS GPU index ✅")
else:
    print("Using FAISS CPU index ✅ (FAISS GPU not available or disabled)")


cpu index ntotal: 18965
Using FAISS GPU index ✅


In [11]:
save_index = index
if USE_FAISS_GPU:
    save_index = faiss.index_gpu_to_cpu(index)

index_path = os.path.join(INDEX_DIR, "faiss_websites.index")
meta_path = os.path.join(INDEX_DIR, "chunks_websites.parquet")

faiss.write_index(save_index, index_path)
chunks_df.to_parquet(meta_path, index=False)

print("saved:", index_path)
print("saved:", meta_path)


saved: D:\data\index\faiss_websites.index
saved: D:\data\index\chunks_websites.parquet


## Ретривал

In [12]:
TOP_K_CHUNKS = int(os.getenv("TOP_K_CHUNKS", "30"))

def search_chunks(query: str, k: int = TOP_K_CHUNKS) -> pd.DataFrame:
    qv = embed_texts([query], is_query=True, batch_size=1)
    faiss.normalize_L2(qv)
    scores, idxs = index.search(qv, k)
    scores = scores[0].tolist()
    idxs = idxs[0].tolist()

    res = chunks_df.iloc[idxs].copy()
    res["score"] = scores
    res["preview"] = res["text"].apply(lambda t: t[:260].replace("\n", " ") + ("..." if len(t) > 260 else ""))
    return res.sort_values("score", ascending=False)

test_q = q_df["query"].iloc[0]
display(search_chunks(test_q, k=10)[["score","web_id","title","url","preview"]].head(10))


,score,web_id,title,url,preview
2975,0.879636,372,Номер расчётного счёта: что это такое и расшиф...,https://alfabank.ru/help/articles/sme/rko/rass...,Номер расчётного счёта: что это такое и расшиф...
4244,0.875997,669,Банк для ООО — банковское обслуживание для орг...,https://alfabank.ru/sme/ooo/,. С нами ваша компания сможет достичь успеха б...
8984,0.874430,1249,Без названия,https://alfabank.ru/m2m/,Без названия Подтвердите запрос пополнения ваш...
11322,0.874153,1704,dogovor_cbo_1082025.pdf,https://alfabank.servicecdn.ru/site-upload/e3/...,. Об изменении номера Счета Банк уведомляет Кл...
13371,0.874153,1705,dogovor_cbo_1072025.pdf,https://alfabank.servicecdn.ru/site-upload/f4/...,. Об изменении номера Счета Банк уведомляет Кл...
5269,0.873625,852,Спецсчета,https://alfabank.ru/corporate/specialaccounts/,Спецсчета Спецсчета Управляйте деньгами подопе...
2977,0.872760,372,Номер расчётного счёта: что это такое и расшиф...,https://alfabank.ru/help/articles/sme/rko/rass...,. Базовый набор документов для открытия счёта ...
8475,0.870568,1157,Как узнать и где посмотреть свой лицевой счёт ...,https://alfabank.ru/help/articles/sme/rko/kak-...,. Для этих целей нужно открывать другой счёт —...
17765,0.870034,1795,partner_cards_service_rules_v53.pdf,https://alfabank.servicecdn.ru/site-upload/43/...,. счета в результате Программа Alfa-Miles (Про...
2980,0.870025,372,Номер расчётного счёта: что это такое и расшиф...,https://alfabank.ru/help/articles/sme/rko/rass...,". Чтобы узнать, в каком банке обслуживается ко..."


In [13]:
def search_docs_dense(query: str, *, k_chunks: int = 80, k_docs: int = 40) -> pd.DataFrame:
    """
    Чистый DENSE doc-level retrieval поверх chunk-level:
    1) достаём top-k чанков
    2) берём лучший чанк на каждый web_id (уникализируем)
    3) сортируем по dense score
    """
    chunks = search_chunks(query, k=k_chunks)
    if chunks is None or len(chunks) == 0:
        return pd.DataFrame(columns=["score","web_id","title","url","preview"])

    chunks = chunks.sort_values("score", ascending=False)
    best = chunks.drop_duplicates(subset=["web_id"], keep="first").copy()

    cols = ["score","web_id","title","url","preview"]
    out = best[cols].head(k_docs).reset_index(drop=True)
    return out


In [14]:
def search_docs(query: str, *, k_chunks: int = 80, k_docs: int = 20) -> pd.DataFrame:
    """
    Doc-level retrieval поверх chunk-level:
    1) достаём top-k чанков
    2) берём лучший чанк на каждый web_id (уникализируем)
    3) возвращаем таблицу в формате: score, web_id, title, url, preview
    """
    chunks = search_chunks(query, k=k_chunks)
    if chunks is None or len(chunks) == 0:
        return pd.DataFrame(columns=["score","web_id","title","url","preview"])

    chunks = chunks.sort_values("score", ascending=False)

    best = chunks.drop_duplicates(subset=["web_id"], keep="first").copy()

    cols = ["score","web_id","title","url","preview"]
    out = best[cols].head(k_docs).reset_index(drop=True)
    return out


In [15]:
import re
import json
import pandas as pd
import numpy as np
from typing import List, Set, Dict, Any, Optional, Tuple

def normalize_results(df) -> pd.DataFrame:
    if df is None:
        df = pd.DataFrame()
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)

    out = df.copy()
    need_cols = ["score", "web_id", "title", "url", "preview"]
    for c in need_cols:
        if c not in out.columns:
            out[c] = ""

    out["score"] = pd.to_numeric(out["score"], errors="coerce").fillna(0.0)
    return out


def retrieval_search_docs(
    query: str,
    *,
    k_chunks: int = 80,
    k_docs: int = 20,
    oversample_docs: int = 40,
) -> Tuple[pd.DataFrame, Optional[str], pd.DataFrame]:
    """
    Возвращает:
    - ranked_df: то, что пойдёт в results (rerank-упорядочено, если есть rerank_score)
    - err
    - dense_df: плотная база для decision (top_score/overlap считаем отсюда)
    """
    q = (query or "").strip()
    try:
        dense_df = normalize_results(search_docs_dense(q, k_chunks=k_chunks, k_docs=max(oversample_docs, k_docs)))
        dense_df = dense_df.sort_values("score", ascending=False).reset_index(drop=True)

        ranked_df = normalize_results(search_docs(q, k_chunks=k_chunks, k_docs=max(oversample_docs, k_docs)))

        if "rerank_score" in ranked_df.columns:
            ranked_df = ranked_df.sort_values("rerank_score", ascending=False)
        else:
            ranked_df = ranked_df.sort_values("score", ascending=False)

        ranked_df = ranked_df.reset_index(drop=True)
        return ranked_df, None, dense_df

    except Exception as e:
        err = f"{type(e).__name__}: {e}"
        return normalize_results(pd.DataFrame()), err, normalize_results(pd.DataFrame())


## Реранкинг

In [16]:
import os
import torch
import pandas as pd
from sentence_transformers import CrossEncoder

RERANK_MODEL = os.getenv("RERANK_MODEL", "cross-encoder/mmarco-mMiniLMv2-L12-H384-v1")
rerank_device = "cuda" if torch.cuda.is_available() else "cpu"
reranker = CrossEncoder(RERANK_MODEL, device=rerank_device)
print("RERANK_MODEL:", RERANK_MODEL, "| device:", rerank_device)

def rerank_docs_df(query: str, df: pd.DataFrame, *, topn: int = 40) -> pd.DataFrame:
    """
    Реранкает doc-level dataframe (score/web_id/title/url/preview).
    Возвращает df с колонкой rerank_score, отсортированный по rerank_score desc.
    """
    if df is None or len(df) == 0:
        return df

    dfx = df.copy().head(topn).reset_index(drop=True)

    cand_texts = (
        dfx["title"].fillna("").astype(str).str.strip()
        + "\n"
        + dfx["preview"].fillna("").astype(str).str.strip().str.slice(0, 900)
    ).tolist()

    pairs = [(query, t) for t in cand_texts]
    scores = reranker.predict(pairs)  # higher = better (обычно)

    dfx["rerank_score"] = scores
    dfx = dfx.sort_values("rerank_score", ascending=False).reset_index(drop=True)

    # добавим хвост (если был) без изменений
    if len(df) > len(dfx):
        tail = df.iloc[len(dfx):].copy()
        tail["rerank_score"] = float("-inf")
        dfx = pd.concat([dfx, tail], ignore_index=True)

    return dfx


RERANK_MODEL: cross-encoder/mmarco-mMiniLMv2-L12-H384-v1 | device: cuda


In [17]:
# 1) сохраняем текущий dense-ретривал (то, что у тебя сейчас называется search_docs)
search_docs_dense = search_docs

# 2) флаги
USE_RERANK = False
RERANK_TOPN = int(os.getenv("RERANK_TOPN", "40"))

# 3) обёртка: dense -> (опционально) rerank
def search_docs(query: str, *, k_chunks: int = 80, k_docs: int = 20) -> pd.DataFrame:
    # Достаём кандидатов чуть больше, чтобы реранкеру было из чего выбирать
    k_docs_fetch = max(k_docs, RERANK_TOPN) if USE_RERANK else k_docs

    df = search_docs_dense(query, k_chunks=k_chunks, k_docs=k_docs_fetch)
    if df is None or len(df) == 0:
        return df

    if USE_RERANK:
        df = rerank_docs_df(query, df, topn=RERANK_TOPN)

    return df.head(k_docs).reset_index(drop=True)

print("search_docs patched ✅ | USE_RERANK =", USE_RERANK, "| RERANK_TOPN =", RERANK_TOPN)


search_docs patched ✅ | USE_RERANK = False | RERANK_TOPN = 40


## Guardrails + Decision layer

In [18]:
# -------------------------
# Конфиги
# -------------------------
BAD_TITLE = {"", "без названия", "untitled"}
BAD_PREVIEW_PATTERNS = ["подтвердите запрос", "captcha", "введите код", "авторизац"]
BAD_URL_PATTERNS = ["private.auth"]

PERSONAL_WEAK_MARKERS = ["у меня", "мне", "мой", "моя", "моё", "моем", "моём"]
PERSONAL_STRONG_MARKERS = [
    "прошу", "договор", "заявлен", "претенз",
    "не начисли", "не приш", "списал", "верните", "остал", "закрыл",
    "оспор", "чарджбек", "возврат", "жалоб"
]

LOGIN_TROUBLE_STRICT = [
    "не получается войти", "не могу войти", "не удается войти", "не удаётся войти",
    "не входит", "не пускает", "ошибка входа",
    "не приходит код", "код не приходит", "смс не приходит", "sms не приходит",
    "заблокирован", "заблокировали"
]

APPEAL_MARKERS = ["обращен", "обращение", "ответ по обращ", "статус обращ", "заявк", "статус заяв", "претенз", "жалоб"]
APPEAL_PERSONAL_HINTS = ["статус", "ответ", "по обращ", "по заяв", "номер", "мой", "моя", "моё", "мне", "у меня"]

NUMBER_PERSONAL_CONTEXT = ["страх", "полис", "сертификат", "договор", "заявк", "кредит", "ипотек", "карта", "счет", "счёт"]

NOTIF_TROUBLE_STRICT = [
    "не приходит уведом", "не приходят уведом", "уведомления нет", "уведомлений нет",
    "не приходят пуш", "не приходит пуш", "пуш не приходит", "push не приходит",
    "не показываются уведомления", "нет пушей",
    "смс вместо пуш", "только смс", "только sms",
    "в уведомлениях нет", "уведомлениях нет",
]

VAGUE_MARKERS = ["там", "это", "вот", "оно", "такое", "то есть", "по идее"]

STOPWORDS_RU = {
    "что", "как", "где", "когда", "почему", "зачем",
    "это", "вот", "там", "ли", "или",
    "услуга", "услуги", "вопрос", "ответ",
    "откуда", "взялась", "появилась", "можно", "нужно",
    "то", "есть", "по", "идее", "вообще", "просто"
}

QUESTION_WORDS = {"что", "как", "где", "когда", "почему", "зачем", "можно", "нужно", "сколько", "куда"}

BRAND_WORDS = {"альфа", "альфабанк", "alfabank", "альфа-банк"}
SHORT_IMPORTANT = {"жкх", "ип", "ооо", "инн", "кпп", "мсс", "sms", "смс", "пуш", "push", "код", "пин", "pin", "чек", "вк", "лк"}
TOO_GENERIC = {"доход", "доходы", "деньги", "финансы", "банк", "банка", "банке", "прибыль"}

# Маркеры продуктов / проблем
PRODUCT_MARKERS = ["карта", "кредит", "ипотек", "вклад", "сч", "счет", "счёт", "договор", "полис", "заявк", "заказ"]
PROBLEM_MARKERS = ["не приш", "не начис", "спис", "верн", "ошиб", "долг", "задолж", "просроч", "не работает", "не открыва", "не отображ", "не показывает", "пропал", "исчез"]

# ВАЖНО: процесс/статус (персонально) отдельно от времени (часто FAQ)
PROCESS_MARKERS = ["статус", "на каком шаге", "этап", "завис", "зависло", "готовы документы", "готовы ли документы"]
TIME_WORDS = ["когда", "через сколько", "сколько ждать", "сегодня", "завтра", "вчера"]

# past tense actions (персональные кейсы)
ACTION_MARKERS = [
    "подал", "подала", "оформил", "оформила", "оформлял", "оформляла",
    "заказал", "заказывал", "заказывала",
    "отправил", "отправила", "создал", "создала",
    "закрыл", "закрыла", "закрывал", "закрывала",
    "подключил", "подключила", "подключал", "подключала",
]
FAMILY_MARKERS = ["муж", "жена", "сын", "дочь", "мама", "папа"]

CALLCENTER_MARKERS = ["консульт", "консультац", "звонок", "позвон", "телефон", "горяч", "колл", "call", "контакт", "оператор"]

# идентификаторы кейса (цифры + контекст)
ID_CONTEXT_MARKERS = ["номер", "заявк", "договор", "обращ", "претенз", "полис", "счет", "счёт", "карта"]

def _contains_any(text: str, patterns: List[str]) -> bool:
    t = (text or "").lower()
    return any(p in t for p in patterns)

def _has_question_word(q_low: str) -> bool:
    toks = re.findall(r"[a-zа-яё0-9-]+", q_low)
    return any(t in QUESTION_WORDS for t in toks)

def filter_noise(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or len(df) == 0:
        return df

    out = df.copy()
    out["title"] = out["title"].fillna("").astype(str)
    out["preview"] = out["preview"].fillna("").astype(str)
    out["url"] = out["url"].fillna("").astype(str)

    out["title_norm"] = out["title"].str.strip().str.lower()
    mask_bad_title = out["title_norm"].isin(BAD_TITLE)

    preview_norm = out["preview"].str.lower()
    mask_bad_preview = preview_norm.apply(lambda x: _contains_any(x, BAD_PREVIEW_PATTERNS))

    url_norm = out["url"].str.lower()
    mask_bad_url = url_norm.apply(lambda x: _contains_any(x, BAD_URL_PATTERNS))

    out = out[~(mask_bad_title | mask_bad_preview | mask_bad_url)].copy()
    return out.drop(columns=["title_norm"], errors="ignore")

def extract_keywords(query: str) -> List[str]:
    q = (query or "").lower()
    toks = re.findall(r"[a-zа-яё0-9-]+", q)
    out: List[str] = []
    for t in toks:
        t = t.strip("-")
        if not t:
            continue
        if t in BRAND_WORDS:
            continue
        if t in STOPWORDS_RU:
            continue
        if len(t) >= 5 or t in SHORT_IMPORTANT:
            out.append(t)
    return sorted(set(out))

def _variants_for_key(k: str) -> Set[str]:
    kk = k.lower()
    variants = {kk}
    if kk.startswith("сообщ") or kk.startswith("уведом"):
        variants |= {"сообщ", "сообщени", "уведом", "уведомлен", "push", "пуш", "sms", "смс", "альфа-чек", "альфа чек"}
    if kk.startswith("кэшб") or kk.startswith("кешб") or "cashback" in kk:
        variants |= {"кэшб", "кешб", "cashback"}
    if kk.startswith("автоплат"):
        variants |= {"автоплат", "автоплатеж", "автоплатёж", "автооплат"}
    return variants

def keyword_overlap_max(query: str, df: pd.DataFrame, topn: int = 5) -> float:
    keys = extract_keywords(query)
    if not keys or df is None or len(df) == 0:
        return 0.0

    dfx = df.sort_values("score", ascending=False).head(min(topn, len(df)))
    best = 0.0
    for _, r in dfx.iterrows():
        blob = (str(r.get("title", "")) + " " + str(r.get("preview", ""))).lower()
        matched = 0
        for k in keys:
            if any(v in blob for v in _variants_for_key(k)):
                matched += 1
        best = max(best, matched / len(keys))
    return float(best)

def classify_intent_first(query: str) -> str:
    """
    Близко к первому intent, но без "когда" как автоматического personal.
    """
    q = (query or "").lower().strip()
    keys = extract_keywords(q)
    words = q.split()

    if _contains_any(q, LOGIN_TROUBLE_STRICT):
        return "personal"
    if (("войти" in q) or ("личный кабинет" in q) or ("в лк" in q) or ("вход" in q)) and (("не" in q) or ("ошибк" in q)):
        return "personal"

    # process markers: почти всегда персональный кейс, если рядом продукт/заявка/счет
    if _contains_any(q, PROCESS_MARKERS) and (_contains_any(q, PRODUCT_MARKERS) or "заяв" in q or "счет" in q or "счёт" in q):
        return "personal"

    # actions: past tense + продукт/заявка
    if _contains_any(q, ACTION_MARKERS) and (_contains_any(q, PRODUCT_MARKERS) or "заяв" in q or "счет" in q or "счёт" in q):
        return "personal"

    # family + продукт
    if _contains_any(q, FAMILY_MARKERS) and _contains_any(q, PRODUCT_MARKERS):
        return "personal"

    if len(words) <= 3:
        toks = set(re.findall(r"[a-zа-яё0-9-]+", q))
        toks = {t for t in toks if t not in STOPWORDS_RU}
        if toks and toks.issubset(TOO_GENERIC):
            return "vague"

    if _contains_any(q, NOTIF_TROUBLE_STRICT):
        return "personal"
    has_notif = ("уведом" in q) or ("пуш" in q) or ("push" in q)
    has_sms = ("смс" in q) or ("sms" in q)
    has_neg = ("нет" in q) or ("не приход" in q) or ("не показыва" in q)
    has_contrast = ("но" in q) or ("вместо" in q) or ("только" in q)
    if (has_notif and has_neg) or (has_notif and has_sms and has_contrast):
        return "personal"

    if _contains_any(q, APPEAL_MARKERS) and (_contains_any(q, APPEAL_PERSONAL_HINTS) or re.search(r"\d{3,}", q)):
        return "personal"

    if "номер" in q and _contains_any(q, NUMBER_PERSONAL_CONTEXT):
        return "personal"

    if _contains_any(q, PERSONAL_STRONG_MARKERS):
        return "personal"

    personal_context = ["спис", "начис", "верн", "деньг", "счет", "счёт", "операц", "платеж", "платёж", "договор", "кредит", "ипотек"]
    if _contains_any(q, PERSONAL_WEAK_MARKERS) and _contains_any(q, personal_context):
        return "personal"

    if len(words) <= 3 and len(keys) == 0:
        return "vague"
    if _contains_any(q, VAGUE_MARKERS) and len(keys) <= 1:
        return "vague"

    return "faq"

def _pin_web_id(df: pd.DataFrame, web_id: int) -> Tuple[pd.DataFrame, bool]:
    if df is None or df.empty:
        return df, False
    if "web_id" not in df.columns:
        return df, False
    m = df["web_id"].astype(str) == str(web_id)
    if not m.any():
        return df, False
    top = df[m].copy()
    rest = df[~m].copy()
    return pd.concat([top, rest], ignore_index=True), True

def guardrails_features(
    query: str,
    ranked_df: pd.DataFrame,
    dense_df: pd.DataFrame,
    *,
    retrieval_error: Optional[str] = None,
    k_docs: int = 10,
) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    ranked_df -> для results (после filter_noise/pin)
    dense_df  -> для стабильных мета-фич (top_score/overlap)
    """
    q = (query or "").strip()
    q_low = q.lower()

    ranked_raw = normalize_results(ranked_df)
    ranked_filtered = normalize_results(filter_noise(ranked_raw)) if len(ranked_raw) else ranked_raw
    base_df = ranked_filtered if len(ranked_filtered) else ranked_raw

    # DENSE база для мета (стабильная)
    dense_base = normalize_results(dense_df).sort_values("score", ascending=False) if len(dense_df) else normalize_results(pd.DataFrame())

    keys = extract_keywords(q)
    word_count = len(re.findall(r"[a-zа-яё0-9-]+", q_low))
    has_question_mark = ("?" in q)
    has_question_word = _has_question_word(q_low)

    has_process_markers = any(x in q_low for x in PROCESS_MARKERS)
    has_time_words = any(x in q_low for x in TIME_WORDS)
    has_action_markers = any(x in q_low for x in ACTION_MARKERS)
    has_family_markers = any(x in q_low for x in FAMILY_MARKERS)

    has_product_markers = any(x in q_low for x in PRODUCT_MARKERS)
    has_problem_markers = any(x in q_low for x in PROBLEM_MARKERS)
    has_digits = bool(re.search(r"\d{3,}", q_low))
    has_personal_id_context = bool(has_digits and any(x in q_low for x in ID_CONTEXT_MARKERS))

    is_callcenter = any(x in q_low for x in CALLCENTER_MARKERS)

    intent = classify_intent_first(q)

    # ВАЖНО: top_score/overlap считаем по DENSE-top5, а не по rerank-упорядоченному base_df
    dense_top_score = float(dense_base["score"].iloc[0]) if len(dense_base) else 0.0
    dense_overlap = keyword_overlap_max(q, dense_base, topn=5) if len(dense_base) else 0.0

    pinned_862 = False
    if is_callcenter:
        base_df, pinned_862 = _pin_web_id(base_df, 862)

    needs_context = (word_count <= 3) and (not has_question_mark) and (not has_question_word)
    underspecified = (
        needs_context
        or ((word_count <= 4) and (len(keys) <= 1))
        or (has_problem_markers and not has_product_markers)
    )

    meta = {
        "intent": intent,
        "keys": keys,
        "n_keys": int(len(keys)),
        "word_count": int(word_count),
        "has_question_mark": bool(has_question_mark),
        "has_question_word": bool(has_question_word),

        # используем DENSE значения
        "top_score": float(dense_top_score),
        "overlap": float(dense_overlap),

        "n_raw": int(len(ranked_raw)),
        "n_filtered": int(len(ranked_filtered)),
        "n_base": int(len(base_df)),

        "has_product_markers": bool(has_product_markers),
        "has_problem_markers": bool(has_problem_markers),

        "has_process_markers": bool(has_process_markers),
        "has_time_words": bool(has_time_words),
        "has_action_markers": bool(has_action_markers),
        "has_family_markers": bool(has_family_markers),

        "has_digits": bool(has_digits),
        "has_personal_id_context": bool(has_personal_id_context),

        "needs_context": bool(needs_context),
        "underspecified": bool(underspecified),

        "is_callcenter": bool(is_callcenter),
        "pinned_862": bool(pinned_862),
        "retrieval_error": retrieval_error,
    }

    return base_df.head(k_docs), meta



In [19]:
# === Decision layer: RULES + optional ML ===

from typing import Dict, Any, Tuple
import pandas as pd

# будет обучен/подгружен в offline части
DECISION_MODEL = None  # sklearn model with predict_proba + classes_

def decision_policy_rules(meta: Dict[str, Any]) -> Tuple[str, str]:
    """Твои текущие правила (fallback)."""
    if meta.get("retrieval_error"):
        return "no_answer", f"Ошибка retrieval: {meta['retrieval_error']}. Проверь search_docs."

    intent = meta.get("intent", "faq")
    top_score = float(meta.get("top_score", 0.0))
    overlap = float(meta.get("overlap", 0.0))
    n_base = int(meta.get("n_base", 0))
    n_keys = int(meta.get("n_keys", 0))

    min_score = float(meta.get("min_score", 0.80))
    strong_score = float(meta.get("strong_score", 0.86))
    min_docs = int(meta.get("min_docs", 3))
    min_overlap = float(meta.get("min_overlap", 0.15))

    has_product = bool(meta.get("has_product_markers", False))
    has_problem = bool(meta.get("has_problem_markers", False))

    has_process = bool(meta.get("has_process_markers", False))
    has_action = bool(meta.get("has_action_markers", False))
    has_family = bool(meta.get("has_family_markers", False))
    has_personal_id = bool(meta.get("has_personal_id_context", False))

    needs_context = bool(meta.get("needs_context", False))
    underspecified = bool(meta.get("underspecified", False))

    # 1) intent personal -> need_clarify
    if intent == "personal":
        return "need_clarify", "Похоже на персональный кейс. Уточни продукт/канал/детали (без персональных данных). Ниже — общие материалы."

    # 2) сильные признаки персонального кейса -> need_clarify
    if has_process or has_action or has_family or has_personal_id:
        return "need_clarify", "Похоже, вопрос про конкретный кейс/процесс. Уточни продукт и детали (без персональных данных). Ниже — общие материалы."

    # 3) vague / needs_context
    if intent == "vague" or needs_context:
        if top_score >= strong_score and overlap >= min_overlap and n_base >= min_docs and (n_keys >= 2):
            return "ok", f"Похоже, нашёл по теме (top_score={top_score:.3f}). Если уточнишь детали — будет точнее."
        return "need_clarify", "Запрос короткий/расплывчатый. Уточни продукт/раздел и что именно хочешь найти."

    # 4) слабый retrieval
    if n_base < min_docs or top_score < min_score:
        return "no_answer", f"Недостаточно уверенно (top_score={top_score:.3f}). Попробуй переформулировать."

    if n_keys == 0:
        return "need_clarify", "Не понял ключевые слова запроса. Уточни предмет: услуга/продукт/раздел."

    # 5) OK-GATE
    def ok_gate() -> bool:
        if intent != "faq":
            return False
        if n_base < min_docs:
            return False
        if top_score < strong_score:
            return False
        if overlap < min_overlap:
            return False
        if underspecified and not has_product:
            return False
        if n_keys <= 1 and not has_product:
            return False
        if has_problem and not has_product:
            return False
        return True

    if ok_gate():
        return "ok", f"Нашёл релевантные источники (top_score={top_score:.3f}, overlap={overlap:.2f})."

    return "need_clarify", "Похоже, нужно уточнение (продукт/канал/детали). Ниже — общие материалы по теме."


def _meta_to_row(meta: Dict[str, Any]) -> pd.DataFrame:
    """
    Фичи для ML. ВАЖНО: список должен совпадать с тем, на чём обучал DECISION_MODEL.
    Если обучал на другом наборе — приведи cols к нему.
    """
    cols = [
        "intent","top_score","overlap","n_base","n_keys","word_count",
        "has_question_mark","has_question_word",
        "has_product_markers","has_problem_markers",
        "has_process_markers","has_action_markers","has_family_markers","has_personal_id_context",
        "needs_context","underspecified",
        "is_callcenter","pinned_862",
    ]
    row = {c: meta.get(c, 0) for c in cols}
    row["intent"] = meta.get("intent", "faq")
    return pd.DataFrame([row])


def decision_policy(meta: Dict[str, Any], *, ok_threshold: float = 0.55) -> Tuple[str, str]:
    """
    ЕДИНСТВЕННАЯ decision_policy для прод-использования.
    Логика:
    0) ошибки retrieval
    1) хард-блоки (personal/process/id/etc.) — всегда need_clarify
    2) если ML нет — rules
    3) если ML есть — ML (ok vs need_clarify), при необходимости fallback на rules
    """
    # 0) ошибки retrieval
    if meta.get("retrieval_error"):
        return "no_answer", f"Ошибка retrieval: {meta['retrieval_error']}. Проверь search_docs."

    # 1) хард-правила (не отдаём ML решать персональные кейсы)
    intent = meta.get("intent", "faq")
    if intent == "personal":
        return "need_clarify", "Похоже на персональный кейс. Уточни продукт/канал/детали (без персональных данных)."

    if meta.get("has_process_markers") or meta.get("has_action_markers") or meta.get("has_family_markers") or meta.get("has_personal_id_context"):
        return "need_clarify", "Похоже, вопрос про конкретный кейс/процесс. Уточни продукт и детали (без персональных данных)."

    # 2) если ML модели нет — чисто rules
    if DECISION_MODEL is None:
        return decision_policy_rules(meta)

    # 3) ML-инференс
    try:
        X = _meta_to_row(meta)
        proba = DECISION_MODEL.predict_proba(X)[0]
        classes = list(DECISION_MODEL.classes_)
        p = dict(zip(classes, proba))

        p_ok = float(p.get("ok", 0.0))

        if p_ok >= ok_threshold:
            return "ok", f"ML: ok (p_ok={p_ok:.2f})"
        return "need_clarify", f"ML: need_clarify (p_ok={p_ok:.2f})"

    except Exception as e:
        # если что-то пошло не так (фичи не совпали и т.п.) — не ломаем прод, падаем в rules
        return decision_policy_rules(meta)


def safe_search_docs(
    query: str,
    *,
    k_chunks: int = 80,
    k_docs: int = 10,
    min_score: float = 0.80,
    min_docs: int = 3,
    strong_score: float = 0.86,
    min_overlap: float = 0.15,
) -> Dict[str, Any]:
    """
    Pipeline:
      retrieval_search_docs -> (ranked_df, err, dense_df)
      guardrails_features   -> (topk_df, meta)  # meta считаем по dense, выдачу берём ranked
      decision_policy       -> status/message
    """
    ranked_df, err, dense_df = retrieval_search_docs(
        query,
        k_chunks=k_chunks,
        k_docs=k_docs,
        oversample_docs=max(15, k_docs * 2),
    )

    topk_df, meta = guardrails_features(
        query,
        ranked_df,
        dense_df,
        retrieval_error=err,
        k_docs=k_docs,
    )

    meta.update({
        "min_score": float(min_score),
        "strong_score": float(strong_score),
        "min_docs": int(min_docs),
        "min_overlap": float(min_overlap),
    })

    status, message = decision_policy(meta)

    return {
        "status": status,
        "message": message,
        "results": topk_df,
        "docs": topk_df.to_dict("records"),
        "meta": meta,
    }



## Оценка метрик

In [20]:
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix

def _parse_ids(x) -> List[int]:
    if pd.isna(x):
        return []
    s = str(x).strip()
    if not s or s.lower() == "nan":
        return []
    return [int(t) for t in re.split(r"[,\s]+", s) if t.strip().isdigit()]

def _jsonify(v):
    # чтобы meta нормально ложилась в parquet/csv
    if isinstance(v, (dict, list, tuple, set)):
        return json.dumps(list(v) if isinstance(v, set) else v, ensure_ascii=False)
    return v

def build_gold_runs(
    gold_path: str = "gold_labels.csv",
    *,
    k_docs: int = 20,
    k_chunks: int = 80,
    save_parquet: str = "gold_runs.parquet",
) -> pd.DataFrame:
    gold = pd.read_csv(Path(gold_path))
    gold["gold_ids"] = gold.get("gold_web_ids", "").apply(_parse_ids)

    runs = []
    for _, r in gold.iterrows():
        query = str(r["query"])
        out = safe_search_docs(query, k_docs=k_docs, k_chunks=k_chunks)

        res = out.get("results")
        pred_ids = []
        top_score = 0.0
        if res is not None and len(res):
            tmp = res.copy()
            tmp["score"] = pd.to_numeric(tmp["score"], errors="coerce").fillna(0.0)
            tmp = tmp.sort_values("score", ascending=False)
            pred_ids = [int(x) for x in tmp["web_id"].head(k_docs).tolist() if str(x).isdigit()]
            top_score = float(tmp["score"].iloc[0])

        meta = out.get("meta") or {}
        meta.setdefault("top_score", top_score)

        runs.append({
            "q_id": int(r["q_id"]),
            "query": query,
            "gold_status": r["label_status"],
            "pred_status": out.get("status"),
            "gold_ids": r["gold_ids"],
            "pred_ids": pred_ids,
            "message": out.get("message"),
            **{f"meta_{k}": _jsonify(v) for k, v in meta.items()},
        })

    runs_df = pd.DataFrame(runs)
    if save_parquet:
        runs_df.to_parquet(save_parquet, index=False)
    return runs_df

def recall_at_k(gold_ids, pred_ids, k) -> float:
    if not gold_ids:
        return np.nan
    return float(any(x in pred_ids[:k] for x in gold_ids))

def mrr_at_k(gold_ids, pred_ids, k) -> float:
    if not gold_ids:
        return np.nan
    for i, pid in enumerate(pred_ids[:k], start=1):
        if pid in gold_ids:
            return 1.0 / i
    return 0.0

def eval_runs(
    runs_df: pd.DataFrame,
    *,
    out_errors_csv: str = "gold_errors.csv",
    out_miss_csv: str = "gold_miss_retrieval.csv",
    out_borderline_csv: str = "gold_borderline.csv",
) -> Dict[str, Any]:
    labels = ["ok", "need_clarify", "no_answer"]

    print(classification_report(
        runs_df["gold_status"], runs_df["pred_status"],
        labels=labels, digits=3
    ))
    print(confusion_matrix(
        runs_df["gold_status"], runs_df["pred_status"],
        labels=labels
    ))

    ok_df = runs_df[runs_df["gold_status"] == "ok"].copy()
    for k in [1, 3, 5, 10, 20]:
        ok_df[f"recall@{k}"] = ok_df.apply(lambda x: recall_at_k(x["gold_ids"], x["pred_ids"], k), axis=1)
        ok_df[f"mrr@{k}"] = ok_df.apply(lambda x: mrr_at_k(x["gold_ids"], x["pred_ids"], k), axis=1)

    retrieval_means = ok_df[[c for c in ok_df.columns if c.startswith("recall@") or c.startswith("mrr@")]].mean(numeric_only=True)

    # ошибки статуса
    errors = runs_df[runs_df["gold_status"] != runs_df["pred_status"]].copy()
    errors.to_csv(out_errors_csv, index=False)

    # miss retrieval (только ok)
    miss = ok_df[ok_df["recall@20"] == 0].copy() if "recall@20" in ok_df.columns else ok_df.iloc[0:0].copy()
    miss.to_csv(out_miss_csv, index=False)

    # “пограничные” кейсы — для твоего тюнинга decision
    borderline = runs_df[
        ((runs_df["gold_status"] == "need_clarify") & (runs_df["pred_status"] == "ok")) |
        ((runs_df["gold_status"] == "need_clarify") & (runs_df["pred_status"] == "no_answer")) |
        ((runs_df["gold_status"] == "ok") & (runs_df["pred_status"] != "ok"))
    ].copy()
    borderline.to_csv(out_borderline_csv, index=False)

    return {
        "retrieval_means_ok": retrieval_means,
        "n_errors": int(len(errors)),
        "n_miss_retrieval_ok": int(len(miss)),
        "n_borderline": int(len(borderline)),
    }




In [21]:
runs_df = build_gold_runs("gold_labels.csv", k_docs=20)
stats = eval_runs(runs_df)
stats["retrieval_means_ok"]

              precision    recall  f1-score   support

          ok      0.554     0.756     0.639        41
need_clarify      0.851     0.762     0.804       105
   no_answer      0.000     0.000     0.000         4

    accuracy                          0.740       150
   macro avg      0.468     0.506     0.481       150
weighted avg      0.747     0.740     0.738       150

[[31 10  0]
 [25 80  0]
 [ 0  4  0]]


D:\Conda\envs\nlp-gpu\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Conda\envs\nlp-gpu\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Conda\envs\nlp-gpu\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


recall@1     0.609756
mrr@1        0.609756
recall@3     0.829268
mrr@3        0.715447
recall@5     0.878049
mrr@5        0.727642
recall@10    1.000000
mrr@10       0.744164
recall@20    1.000000
mrr@20       0.744164
dtype: float64

In [22]:
def show_ok_to_need(runs_df, n=50):
    df = runs_df[(runs_df["gold_status"]=="ok") & (runs_df["pred_status"]=="need_clarify")].copy()
    print("ok→need_clarify:", len(df))
    cols = ["q_id","query","pred_status","gold_status","message",
            "meta_intent","meta_top_score","meta_overlap","meta_n_keys",
            "meta_has_process_markers","meta_has_action_markers","meta_has_personal_id_context","meta_needs_context","meta_underspecified"]
    cols = [c for c in cols if c in df.columns]
    display(df[cols].head(n))

show_ok_to_need(runs_df, n=50)


ok→need_clarify: 10


,q_id,query,pred_status,gold_status,message,meta_intent,meta_top_score,meta_overlap,meta_n_keys,meta_has_process_markers,meta_has_action_markers,meta_has_personal_id_context,meta_needs_context,meta_underspecified
0,5287,Почему не начислен кэшбэк 0% за оплату ЖКУ за ...,need_clarify,ok,"Похоже, нужно уточнение (продукт/канал/детали)...",faq,0.874369,0.500000,4,False,False,False,False,True
17,2432,Не приходит sms об операциях Они для меня бес...,need_clarify,ok,"Похоже, нужно уточнение (продукт/канал/детали)...",faq,0.866135,0.000000,4,False,False,False,False,False
26,3067,"Я отключал смс оповещение, почему списана коми...",need_clarify,ok,"Похоже, нужно уточнение (продукт/канал/детали)...",faq,0.846275,0.400000,5,False,False,False,False,True
55,1220,"Ранее у меня было приложение alfa pay, сейчас ...",need_clarify,ok,"Похоже, нужно уточнение (продукт/канал/детали)...",faq,0.906841,0.142857,14,False,False,False,False,False
74,3631,Хочу снять ограничения по платежам,need_clarify,ok,"Похоже, нужно уточнение (продукт/канал/детали)...",faq,0.873058,0.000000,3,False,False,False,False,False
88,1478,"Добрый день, заказывал новую карту, хочу узнат...",need_clarify,ok,Похоже на персональный кейс. Уточни продукт/ка...,personal,0.881303,0.428571,7,False,True,False,False,False
107,643,Кэшбэк альфа Трэвел,need_clarify,ok,Запрос короткий/расплывчатый. Уточни продукт/р...,faq,0.856715,0.500000,2,False,False,False,True,True
112,5079,подключить сбп,need_clarify,ok,Запрос короткий/расплывчатый. Уточни продукт/р...,faq,0.870177,1.000000,1,False,False,False,True,True
116,5007,"Отключить все уведомления , и вернуть деньги з...",need_clarify,ok,"Похоже, нужно уточнение (продукт/канал/детали)...",faq,0.874740,0.200000,5,False,False,False,False,True
136,713,Где я могу найти БИК банка ?,need_clarify,ok,"Похоже, нужно уточнение (продукт/канал/детали)...",faq,0.847707,1.000000,2,False,False,False,False,False


In [23]:
gold = pd.read_csv("gold_labels.csv")
gold["gold_ids"] = gold.get("gold_web_ids", "").apply(_parse_ids)

ok = gold[gold["label_status"] == "ok"].copy()

def _eval_retrieval(df_ok: pd.DataFrame, mode: str, k_chunks: int = 80, k_docs: int = 40):
    recalls = {1: [], 3: [], 5: []}
    mrrs = {1: [], 3: [], 5: []}

    for _, r in df_ok.iterrows():
        q = str(r["query"])
        gold_ids = r["gold_ids"]

        if mode == "dense":
            res = search_docs_dense(q, k_chunks=k_chunks, k_docs=k_docs)
            res = normalize_results(res).sort_values("score", ascending=False)
        else:  # "rerank"
            res = search_docs(q, k_chunks=k_chunks, k_docs=k_docs)
            res = normalize_results(res)
            if "rerank_score" in res.columns:
                res = res.sort_values("rerank_score", ascending=False)
            else:
                res = res.sort_values("score", ascending=False)

        pred_ids = [int(x) for x in res["web_id"].tolist() if str(x).isdigit()]

        for k in [1, 3, 5]:
            recalls[k].append(recall_at_k(gold_ids, pred_ids, k))
            mrrs[k].append(mrr_at_k(gold_ids, pred_ids, k))

    out = {
        "mode": mode,
        "recall@1": float(np.nanmean(recalls[1])),
        "mrr@1": float(np.nanmean(mrrs[1])),
        "recall@3": float(np.nanmean(recalls[3])),
        "mrr@3": float(np.nanmean(mrrs[3])),
        "recall@5": float(np.nanmean(recalls[5])),
        "mrr@5": float(np.nanmean(mrrs[5])),
    }
    return out

USE_RERANK = False
dense_metrics = _eval_retrieval(ok, "dense")

USE_RERANK = True
rerank_metrics = _eval_retrieval(ok, "rerank")

pd.DataFrame([dense_metrics, rerank_metrics])





,mode,recall@1,mrr@1,recall@3,mrr@3,recall@5,mrr@5
0,dense,0.585366,0.585366,0.829268,0.703252,0.878049,0.713008
1,rerank,0.463415,0.463415,0.658537,0.544715,0.804878,0.577642


## Обучение модели для Decision-layer

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

def train_decision_model_from_runs(runs_df: pd.DataFrame, *, merge_no_answer=True, test_size=0.2, random_state=42):
    df = runs_df.copy()
    y = df["gold_status"].astype(str)
    if merge_no_answer:
        y = y.replace({"no_answer": "need_clarify"})

    # фичи из meta_*
    feat = pd.DataFrame({
        "intent": df.get("meta_intent", "faq").astype(str),
        "top_score": pd.to_numeric(df.get("meta_top_score", 0), errors="coerce").fillna(0.0),
        "overlap": pd.to_numeric(df.get("meta_overlap", 0), errors="coerce").fillna(0.0),
        "n_base": pd.to_numeric(df.get("meta_n_base", 0), errors="coerce").fillna(0),
        "n_keys": pd.to_numeric(df.get("meta_n_keys", 0), errors="coerce").fillna(0),
        "word_count": pd.to_numeric(df.get("meta_word_count", 0), errors="coerce").fillna(0),
        "has_question_mark": pd.to_numeric(df.get("meta_has_question_mark", 0), errors="coerce").fillna(0).astype(int),
        "has_question_word": pd.to_numeric(df.get("meta_has_question_word", 0), errors="coerce").fillna(0).astype(int),
        "has_product_markers": pd.to_numeric(df.get("meta_has_product_markers", 0), errors="coerce").fillna(0).astype(int),
        "has_problem_markers": pd.to_numeric(df.get("meta_has_problem_markers", 0), errors="coerce").fillna(0).astype(int),
        "has_process_markers": pd.to_numeric(df.get("meta_has_process_markers", 0), errors="coerce").fillna(0).astype(int),
        "has_action_markers": pd.to_numeric(df.get("meta_has_action_markers", 0), errors="coerce").fillna(0).astype(int),
        "has_family_markers": pd.to_numeric(df.get("meta_has_family_markers", 0), errors="coerce").fillna(0).astype(int),
        "has_personal_id_context": pd.to_numeric(df.get("meta_has_personal_id_context", 0), errors="coerce").fillna(0).astype(int),
        "needs_context": pd.to_numeric(df.get("meta_needs_context", 0), errors="coerce").fillna(0).astype(int),
        "underspecified": pd.to_numeric(df.get("meta_underspecified", 0), errors="coerce").fillna(0).astype(int),
        "is_callcenter": pd.to_numeric(df.get("meta_is_callcenter", 0), errors="coerce").fillna(0).astype(int),
        "pinned_862": pd.to_numeric(df.get("meta_pinned_862", 0), errors="coerce").fillna(0).astype(int),
    })

    X_train, X_test, y_train, y_test = train_test_split(
        feat, y, test_size=test_size, random_state=random_state, stratify=y
    )

    cat_cols = ["intent"]
    num_cols = [c for c in feat.columns if c not in cat_cols]

    pre = ColumnTransformer([
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore")),
        ]), cat_cols),
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
        ]), num_cols),
    ])

    clf = LogisticRegression(max_iter=800, class_weight="balanced", n_jobs=-1)

    model = Pipeline([("pre", pre), ("clf", clf)])
    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    print("=== Decision model holdout ===")
    print(classification_report(y_test, pred, digits=3))
    print(confusion_matrix(y_test, pred, labels=sorted(y.unique())))

    return model

# обучаем и кладём в глобальную переменную decision layer
DECISION_MODEL = train_decision_model_from_runs(runs_df, merge_no_answer=True)
print("DECISION_MODEL ready ✅")


=== Decision model holdout ===
              precision    recall  f1-score   support

need_clarify      0.826     0.864     0.844        22
          ok      0.571     0.500     0.533         8

    accuracy                          0.767        30
   macro avg      0.699     0.682     0.689        30
weighted avg      0.758     0.767     0.761        30

[[19  3]
 [ 4  4]]
DECISION_MODEL ready ✅


In [25]:
import joblib
joblib.dump(DECISION_MODEL, "decision_model.joblib")
print("saved decision_model.joblib ✅")


saved decision_model.joblib ✅


## Ручное тестирование вопросов, ввод LLM в ответы

In [26]:
import re
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional

import requests

# --- OLLAMA CONFIG ---
OLLAMA_URL = "http://127.0.0.1:11434"
OLLAMA_MODEL = "qwen2.5:3b"

DEFAULT_SYSTEM = (
    "Ты аккуратный помощник банка. Не выдумывай факты. "
    "Если данных нет — честно скажи об этом и предложи уточнить."
)

def ollama_generate(
    prompt: str,
    *,
    model: str = OLLAMA_MODEL,
    system: str = DEFAULT_SYSTEM,
    temperature: float = 0.2,
    timeout: int = 180,
) -> str:
    """
    Вызов Ollama /api/generate (проще и стабильнее для одиночных промптов).
    """
    payload = {
        "model": model,
        "prompt": prompt,
        "system": system,
        "stream": False,
        "options": {
            "temperature": float(temperature),
        },
    }
    r = requests.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=timeout)
    r.raise_for_status()
    data = r.json()
    return (data.get("response") or "").strip()

def build_context_from_docs(
    docs: List[Dict[str, Any]],
    *,
    max_docs: int = 6,
    max_chars: int = 9000,
) -> str:
    """
    Контекст для RAG: title + url + preview.
    """
    parts = []
    total = 0
    for d in (docs or [])[:max_docs]:
        title = str(d.get("title", "")).strip()
        url = str(d.get("url", "")).strip()
        preview = str(d.get("preview", "")).strip()
        chunk = f"- {title}\n  {url}\n  {preview}\n"
        if total + len(chunk) > max_chars:
            break
        parts.append(chunk)
        total += len(chunk)
    return "\n".join(parts).strip()



In [27]:
from sentence_transformers import SentenceTransformer

# Можно заменить на другую модель (e5-base даст качество лучше, но тяжелее)
E5_MODEL_NAME = "intfloat/multilingual-e5-small"
e5_model = SentenceTransformer(E5_MODEL_NAME)

def embed_fn(texts: List[str]) -> np.ndarray:
    """
    Для E5 обязательно префиксы query:/passage:
    Здесь используем query:, потому что сравниваем запросы с запросами (clarify bank).
    """
    texts = [f"query: {str(t)}" for t in texts]
    vecs = e5_model.encode(
        texts,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    return np.asarray(vecs, dtype="float32")


In [28]:
#clarify_bank = build_clarify_bank("gold_labels.csv", embed_fn=embed_fn)

In [29]:
def _parse_clarify_note(note: str) -> Dict[str, Any]:
    """
    note format:
      "ask_details|ask_product::Уточните ..."
      или просто "Уточните ..."
    """
    s = (note or "").strip()
    if not s:
        return {"codes": [], "text": ""}

    if "::" in s:
        left, right = s.split("::", 1)
        codes = [c.strip() for c in left.split("|") if c.strip()]
        text = right.strip()
        return {"codes": codes, "text": text}

    return {"codes": [], "text": s}

def build_clarify_bank_v2(gold_path: str, embed_fn):
    gold = pd.read_csv(Path(gold_path))
    df = gold[(gold["label_status"] == "need_clarify") & gold["clarify_note"].notna()].copy()

    df["query"] = df["query"].astype(str)
    df["clarify_note"] = df["clarify_note"].astype(str)

    parsed = df["clarify_note"].apply(_parse_clarify_note)
    df["note_codes"] = parsed.apply(lambda x: x["codes"])
    df["note_text"]  = parsed.apply(lambda x: x["text"])

    texts = df["query"].tolist()
    E = embed_fn(texts).astype("float32")

    return {
        "E": E,
        "texts": texts,
        "note_texts": df["note_text"].tolist(),
        "note_codes": df["note_codes"].tolist(),
    }

# BUILD BANK (важно выполнить, иначе будет NameError)
clarify_bank = build_clarify_bank_v2("gold_labels.csv", embed_fn=embed_fn)

print("clarify_bank size:", len(clarify_bank["texts"]))


clarify_bank size: 105


In [30]:
def pick_clarify_hint_topk(
    query: str,
    bank: dict,
    embed_fn,
    *,
    topk: int = 5,
    min_sim: float = 0.70,
    max_examples: int = 3,
) -> Optional[Dict[str, Any]]:
    """
    Возвращает:
      {
        "codes": [...],        # объединение кодов из топ-k
        "examples": [...],     # 1-3 примера (только matched_query + sim)
        "top_sim": float,
        "best_text": str,      # лучший note_text (для режима gold)
        "best_query": str
      }
    """
    qE = embed_fn([query]).astype("float32")[0]
    sims = bank["E"] @ qE

    idx = np.argsort(-sims)[:max(topk, 1)]
    picked = []
    for i in idx:
        sim = float(sims[i])
        if sim < min_sim:
            continue
        picked.append((int(i), sim))

    if not picked:
        return None

    best_i, best_sim = picked[0]
    best_text = bank["note_texts"][best_i]
    best_query = bank["texts"][best_i]

    # aggregate codes
    codes = []
    for i, sim in picked:
        for c in bank["note_codes"][i]:
            if c and c not in codes:
                codes.append(c)

    # examples (только queries, чтобы не копировать твои тексты 1-в-1)
    examples = []
    for i, sim in picked[:max_examples]:
        examples.append({
            "sim": sim,
            "matched_query": bank["texts"][i],
        })

    return {
        "codes": codes[:8],
        "examples": examples,
        "top_sim": float(best_sim),
        "best_text": best_text,
        "best_query": best_query,
    }


In [31]:
MONEY_MISSING_PATTERNS = [
    "где мои деньги", "куда делись деньги", "куда ушли деньги", "пропали деньги",
    "не вижу деньги", "исчезли деньги", "деньги пропали",
    "не пришли деньги", "не поступили деньги",
]
APP_RATE_PATTERNS = [
    "оценить приложение", "приложение оценить", "оценка приложения", "поставить оценку",
    "оставить отзыв", "отзыв о приложении", "рейтинг приложения", "звезды", "звёзды"
]

CODE_GUIDE_V2 = {
    "ask_what_happened": "что именно произошло (не поступило / списалось / не видно / в обработке)",
    "ask_what_exactly":  "что именно хотите оценить (приложение в целом / конкретную функцию / отзыв в магазине приложений)",
    "ask_product":  "какой продукт/сервис (карта/счёт/вклад/кредит/инвестиции/уведомления и т.д.)",
    "ask_channel":  "где это происходит (приложение/сайт/чат/банкомат) и если про уведомления — какой канал (push/sms/почта)",
    "ask_time":     "когда это случилось (дата/примерное время)",
    "ask_amount":   "на какую сумму (примерно) и в какой валюте",
    "ask_details":  "каких деталей не хватает, чтобы понять вопрос",
    "how_to_do":    "что именно хотите сделать (подключить/отключить/изменить/оформить)",
    "how_to_check": "что и где проверяете (какой экран/раздел/статус)",
    "route_support":"если это персональный случай — уточнить контекст и при необходимости направить в поддержку",
}

def derive_clarify_tags(query: str, meta: Dict[str, Any], hint_codes: List[str]) -> List[str]:
    q = (query or "").lower().strip()
    meta = meta or {}
    tags = set(hint_codes or [])

    # базовые сигналы из meta
    if not meta.get("has_product_markers", False):
        tags.add("ask_product")
    if meta.get("needs_context", False) or meta.get("underspecified", False):
        tags.add("ask_details")

    # доменные эвристики
    if any(p in q for p in MONEY_MISSING_PATTERNS) or ("где" in q and "деньг" in q):
        tags |= {"ask_what_happened", "ask_product", "ask_channel", "ask_time", "ask_amount"}

    if any(p in q for p in APP_RATE_PATTERNS) or ("оцен" in q and "прилож" in q):
        tags |= {"ask_what_exactly", "ask_channel"}

    # порядок важности
    priority = [
        "ask_what_happened", "ask_what_exactly",
        "ask_product", "ask_channel", "ask_time", "ask_amount",
        "ask_details", "how_to_do", "how_to_check", "route_support"
    ]
    ordered = [t for t in priority if t in tags]
    for t in tags:
        if t not in ordered:
            ordered.append(t)
    return ordered


In [32]:
#clarify_bank = build_clarify_bank_v2("gold_labels.csv", embed_fn)


In [33]:
def _postprocess_no_extra_numbers(text: str, query: str) -> str:
    """
    Если в query нет цифр/ноля — убираем внезапные цифры и "оценка 0".
    """
    q = (query or "").lower()
    has_digits_in_query = bool(re.search(r"\d", q)) or ("нол" in q)
    out = text or ""
    if not has_digits_in_query:
        out = re.sub(r'(?i)\bоценк\w*\s*0\b', "оценку", out)
        out = re.sub(r'(?<!\w)\d+(?!\w)', "", out)
        out = re.sub(r"\s{2,}", " ", out).strip()
    return out

def _to_second_person_ru(t: str) -> str:
    if not t:
        return t
    repl = [
        (r"\bмоим\b", "вашим"),
        (r"\bмоем\b", "вашем"),
        (r"\bмоём\b", "вашем"),
        (r"\bмоей\b", "вашей"),
        (r"\bмоего\b", "вашего"),
        (r"\bмоих\b", "ваших"),
        (r"\bмоему\b", "вашему"),
        (r"\bмоими\b", "вашими"),
        (r"\bмои\b", "ваши"),
        (r"\bмой\b", "ваш"),
        (r"\bмоя\b", "ваша"),
        (r"\bмоё\b", "ваше"),
    ]
    out = t
    for pat, rep in repl:
        out = re.sub(pat, rep, out, flags=re.IGNORECASE)
    return out

def _clean_llm_text(txt: str) -> str:
    """
    Приводим ответ к формату:
      1 строка вопрос
      2-4 строки буллеты "— "
    """
    t = (txt or "").strip()

    # убрать "Запрос:" / "Вопрос:" если проскочило
    t = re.sub(r"(?im)^\s*запрос\s*:\s*.*$", "", t).strip()
    t = re.sub(r"(?im)^\s*вопрос\s*:\s*", "", t).strip()

    lines = [ln.strip() for ln in t.splitlines() if ln.strip()]
    if not lines:
        return ""

    # нормализуем строки
    norm = []
    for ln in lines:
        ln = re.sub(r"^\d+\)\s*", "", ln)          # убрать нумерацию
        ln = re.sub(r"^[-•]\s*", "— ", ln)         # буллеты
        ln = re.sub(r"^[—\s]+", "", ln)            # пока уберём тире — потом добавим где нужно
        norm.append(ln.strip())

    # первая строка = вопрос
    qline = norm[0]
    if not qline.endswith("?"):
        qline = qline.rstrip(".") + "?"

    bullets = norm[1:]
    # оставим максимум 4 буллета
    bullets = bullets[:4]

    # если модель не дала буллеты — пусть будет пусто (позже сделаем fallback)
    out_lines = [qline]
    for b in bullets:
        if b:
            out_lines.append("— " + b)

    t = "\n".join(out_lines).strip()
    t = _to_second_person_ru(t)
    return t

def _fallback_clarify(query: str, tags: List[str]) -> str:
    """
    На случай если LLM выдал мусор/пусто: делаем стабильный шаблон по tags.
    """
    tags = tags or ["ask_details", "ask_product"]
    bullets = []
    for t in tags:
        if t in CODE_GUIDE_V2:
            bullets.append(f"— {CODE_GUIDE_V2[t]}")
        if len(bullets) >= 3:
            break
    return "Уточните, пожалуйста, детали по вашему запросу?\n" + "\n".join(bullets)


In [34]:
def llm_answer_ok(query: str, docs: List[Dict[str, Any]], *, model: str) -> str:
    context = build_context_from_docs(docs, max_docs=6)

    prompt = f"""Вопрос пользователя: {query}

Источники (выдержки):
{context}

Задача:
1) Ответь по сути, опираясь ТОЛЬКО на источники.
2) Если в источниках нет ответа — честно скажи, что в базе нет точного ответа, и задай 1-2 уточняющих вопроса.
3) В конце коротко перечисли 2-4 источника (title или url), на которые опирался.
"""
    txt = ollama_generate(prompt, model=model, temperature=0.2)
    return txt.strip()




In [35]:
from typing import Dict, Any

def ask_service(
    query: str,
    *,
    k_docs: int = 8,
    clarify_bank: Optional[dict] = None,
    embed_fn=None,
    clarify_min_sim: float = 0.70,
    clarify_topk: int = 5,
    clarify_mode: str = "llm",       # "off" | "gold" | "llm" | "gold_then_llm"
    use_llm: bool = True,
    llm_model: str = OLLAMA_MODEL,
    llm_for: str = "both",           # "need_clarify" | "ok" | "both"
    **kwargs
) -> Dict[str, Any]:
    out = safe_search_docs(query, k_docs=k_docs, **kwargs)
    status = out.get("status")
    msg = out.get("message", "")
    docs = out.get("docs", []) or []
    meta = out.get("meta", {}) or {}

    hint = None
    if status == "need_clarify" and clarify_bank is not None and embed_fn is not None:
        hint = pick_clarify_hint_topk(
            query,
            clarify_bank,
            embed_fn,
            topk=clarify_topk,
            min_sim=clarify_min_sim,
            max_examples=3
        )

    final_msg = msg

    if status == "need_clarify":
        hint_codes = (hint.get("codes") if hint else []) or []
        best_text  = (hint.get("best_text") if hint else "") or ""

        if clarify_mode == "off":
            final_msg = msg

        elif clarify_mode == "gold":
            final_msg = best_text or msg
            final_msg = _to_second_person_ru(final_msg)  # на всякий
            if final_msg and not final_msg.endswith("?") and "\n" not in final_msg:
                final_msg = final_msg.rstrip(".") + "?"

        elif clarify_mode == "llm":
            if use_llm and llm_for in {"need_clarify", "both"}:
                final_msg = llm_rewrite_need_clarify(
                    query,
                    hint_codes=hint_codes,
                    meta=meta,
                    hint=hint,
                    model=llm_model
                )
            else:
                final_msg = msg

        elif clarify_mode == "gold_then_llm":
            if use_llm and llm_for in {"need_clarify", "both"}:
                final_msg = llm_rewrite_need_clarify(
                    query,
                    hint_codes=hint_codes,
                    meta=meta,
                    hint=hint,
                    model=llm_model
                )
            else:
                final_msg = best_text or msg

        else:
            final_msg = msg

    elif status == "ok":
        if use_llm and llm_for in {"ok", "both"}:
            try:
                final_msg = llm_answer_ok(query, docs, model=llm_model)
            except Exception as e:
                final_msg = msg + f"\n\n(LLM error: {type(e).__name__}: {e})"
        else:
            final_msg = msg

    # pretty print
    print("\n=== QUERY ===")
    print(query)
    print("\n=== STATUS ===")
    print(f"[{status}] {final_msg}")

    print("\n=== TOP DOCS ===")
    df = out.get("results")
    if isinstance(df, pd.DataFrame) and len(df):
        display(df[["score", "web_id", "title", "url"]].head(k_docs))

    return {"out": out, "final": final_msg, "hint": hint}


In [36]:
import re
from typing import List, Dict, Any, Optional

# 1) Добавим паттерн для "госуведомления" (чтобы теги были адекватнее)
GOS_NOTIF_PATTERNS = ["госуведомления", "гос уведомления", "гос-уведомления"]

def derive_clarify_tags(query: str, meta: Dict[str, Any], hint_codes: List[str]) -> List[str]:
    q = (query or "").lower().strip()
    meta = meta or {}
    tags = set(hint_codes or [])

    if not meta.get("has_product_markers", False):
        tags.add("ask_product")
    if meta.get("needs_context", False) or meta.get("underspecified", False):
        tags.add("ask_details")

    if any(p in q for p in MONEY_MISSING_PATTERNS) or ("где" in q and "деньг" in q):
        tags |= {"ask_what_happened", "ask_product", "ask_channel", "ask_time", "ask_amount"}

    if any(p in q for p in APP_RATE_PATTERNS) or ("оцен" in q and "прилож" in q):
        tags |= {"ask_what_exactly", "ask_channel"}

    if any(p in q for p in GOS_NOTIF_PATTERNS):
        tags |= {"ask_product", "ask_channel", "how_to_do"}

    priority = [
        "ask_what_happened", "ask_what_exactly",
        "ask_product", "ask_channel", "ask_time", "ask_amount",
        "ask_details", "how_to_do", "how_to_check", "route_support"
    ]
    ordered = [t for t in priority if t in tags]
    for t in tags:
        if t not in ordered:
            ordered.append(t)
    return ordered


# 2) Санитайзер: выкидываем опасные/мусорные строки (профиль/данные/ask_*)
SENSITIVE_LINE_PATTERNS = [
    r"детал(и|ей)\s+вашего\s+профил",   # "детали вашего профиля"
    r"данн(ые|ых)\s+профил",           # "данные профиля"
    r"номер\s+(карты|сч[её]та)",       # "номер карты/счета"
    r"\bcvv\b|\bcvc\b|\bпин\b",        # CVV/PIN
    r"паспорт|снилс|инн|код\s+из\s+смс|одноразов",
    r"логин|парол",
]

def _strip_trailing_dashes(s: str) -> str:
    return re.sub(r"[\s—\-]+$", "", s).strip()

def _clean_llm_text_v4(txt: str, tags: List[str]) -> str:
    """
    Приводим к формату:
      1 строка вопрос (с ?)
      2–4 буллета "— ..."
    + убираем "—?" / хвосты / пустые буллеты / ask_* строки
    """
    t = (txt or "").strip()

    # убрать "Запрос:" / "Вопрос:" если проскочило
    t = re.sub(r"(?im)^\s*запрос\s*:\s*.*$", "", t).strip()
    t = re.sub(r"(?im)^\s*вопрос\s*:\s*", "", t).strip()

    lines = [ln.strip() for ln in t.splitlines() if ln.strip()]
    if not lines:
        return ""

    # нормализация линий
    norm = []
    for ln in lines:
        ln = ln.replace("—?", "?").replace("-?", "?")
        ln = _strip_trailing_dashes(ln)

        # уберём маркеры буллетов/нумерацию
        ln = re.sub(r"^\d+\)\s*", "", ln)
        ln = re.sub(r"^[-•]\s*", "", ln)
        ln = re.sub(r"^—\s*", "", ln)

        ln = ln.strip()
        if not ln:
            continue

        # фильтр на чувствительные/нежелательные строки
        low = ln.lower()
        if "ask_" in low:
            continue
        bad = any(re.search(p, low) for p in SENSITIVE_LINE_PATTERNS)
        if bad:
            continue

        norm.append(ln)

    if not norm:
        return ""

    # первая строка — вопрос
    qline = norm[0]
    qline = qline.replace("—", " ").replace("  ", " ").strip()
    qline = _strip_trailing_dashes(qline)
    if not qline.endswith("?"):
        qline = qline.rstrip(".") + "?"

    # остальные — буллеты
    bullets_raw = norm[1:]
    bullets = []
    for b in bullets_raw:
        b = b.replace("—", " ").strip()
        b = _strip_trailing_dashes(b)
        if not b:
            continue
        # выкинем "общие рассуждения про цели/результаты"
        if re.search(r"цели|планируете|результат", b.lower()):
            continue
        bullets.append(b)

    bullets = bullets[:4]

    # если буллеты пустые — сделаем осмысленные из tags
    if len(bullets) == 0:
        bullets = []
        for tg in (tags or []):
            if tg in CODE_GUIDE_V2:
                bullets.append(CODE_GUIDE_V2[tg])
            if len(bullets) >= 3:
                break

    out_lines = [qline]
    for b in bullets[:4]:
        out_lines.append("— " + b)

    out = "\n".join(out_lines).strip()
    out = _to_second_person_ru(out)
    return out


def _fallback_clarify_v2(query: str, tags: List[str]) -> str:
    q = (query or "").lower()

    # спец-фоллбеки под частые случаи
    if any(p in q for p in MONEY_MISSING_PATTERNS) or ("где" in q and "деньг" in q):
        return (
            "Что именно произошло с деньгами: они не поступили, списались или вы не видите баланс?\n"
            "— К какому продукту это относится (карта/счёт/вклад/инвестиции)?\n"
            "— Где это видно (приложение/сайт/банкомат) и что именно вы видите?\n"
            "— Когда и на какую сумму была операция (примерно)?"
        )

    if any(p in q for p in GOS_NOTIF_PATTERNS):
        return (
            "Что вы имеете в виду под «госуведомлениями»?\n"
            "— Где вы это видите (приложение/сайт/чат) и какой канал уведомлений (push/SMS/почта)?\n"
            "— Что именно хотите сделать: подключить, отключить или найти настройки?\n"
            "— Уведомления относятся к Госуслугам/госорганам или к сервису внутри банка?"
        )

    # общий fallback по tags
    bullets = []
    for tg in (tags or ["ask_details", "ask_product"]):
        if tg in CODE_GUIDE_V2:
            bullets.append("— " + CODE_GUIDE_V2[tg])
        if len(bullets) >= 3:
            break
    return "Уточните, пожалуйста, детали по вашему запросу?\n" + "\n".join(bullets)


# 3) Переписываем LLM-уточнение: НЕ показываем коды ask_*, только смыслы
def llm_rewrite_need_clarify(
    query: str,
    *,
    hint_codes: List[str],
    meta: Dict[str, Any],
    hint: Optional[Dict[str, Any]],
    model: str,
) -> str:
    tags = derive_clarify_tags(query, meta or {}, hint_codes or [])

    # показываем LLM только человеческие смыслы, без "ask_product"
    guide_lines = []
    for c in tags:
        if c in CODE_GUIDE_V2:
            guide_lines.append(f"- {CODE_GUIDE_V2[c]}")
    guide = "\n".join(guide_lines) if guide_lines else "- уточните, что именно вы имеете в виду"

    # примеры — только похожие запросы (без твоих готовых текстов)
    examples_block = ""
    if hint and hint.get("examples"):
        ex = [e.get("matched_query") for e in hint["examples"] if e.get("matched_query")]
        ex = ex[:3]
        if ex:
            examples_block = "\n\nПохожие запросы (только для ориентира):\n" + "\n".join([f"- {x}" for x in ex])

    prompt = f"""Ты — помощник банка. Сформулируй уточняющий вопрос к запросу пользователя.

ЖЁСТКИЕ правила:
- Пиши по-русски, естественно.
- Всегда обращайся к пользователю на «вы», используй «ваш/ваша/ваше».
- Запрещено использовать «мой/моя/моё/мои».
- Не придумывай факты/числа.
- Не проси персональные данные (ФИО, номер карты/счёта, телефон, паспорт, коды из СМС).
- Не пиши слова/метки вида "ask_product", "ask_channel" и т.п.
- Не спрашивай про «цели», «планируете», «для каких результатов».

Формат ответа строго:
1) ОДНО короткое предложение-вопрос (заканчивается '?')
2) Затем 2–4 буллета, каждый начинается с "— "

Запрос пользователя: {query}

Что важно уточнить:
{guide}{examples_block}

Сгенерируй уточнение в этом формате.
"""
    try:
        raw = ollama_generate(prompt, model=model, temperature=0.1)
        cleaned = _clean_llm_text_v4(raw, tags)
        cleaned = _postprocess_no_extra_numbers(cleaned, query)

        # если всё равно мусор — fallback
        if (not cleaned) or ("ask_" in cleaned.lower()):
            return _fallback_clarify_v2(query, tags)

        return cleaned
    except Exception:
        return _fallback_clarify_v2(query, tags)


In [37]:
# ask_service("Госуведомления", clarify_bank=clarify_bank, embed_fn=embed_fn,
#             use_llm=True, llm_for="need_clarify", clarify_mode="llm", clarify_min_sim=0.70)

ask_service("Почему мне приходят уведомления?", clarify_bank=clarify_bank, embed_fn=embed_fn,
            use_llm=True, llm_for="need_clarify", clarify_mode="llm", clarify_min_sim=0.70)




=== QUERY ===
Почему мне приходят уведомления?

=== STATUS ===
[need_clarify] Здравствуйте, вы можете добавить, какой продукт/сервис (карта/счёт/вклад/кредит/инвестиции) и где это происходит (приложение/сайт/чат/банкомат)? Каких деталей не хватает, чтобы понять ваш вопрос?
— какой продукт/сервис (карта/счёт/вклад/кредит/инвестиции/уведомления и т.д.)
— где это происходит (приложение/сайт/чат/банкомат) и если про уведомления — какой канал (push/sms/почта)
— каких деталей не хватает, чтобы понять вопрос

=== TOP DOCS ===


,score,web_id,title,url
0,0.845569,395,Как заполнить платежное поручение в 2025 году ...,https://alfabank.ru/help/articles/sme/start/pl...
1,0.855647,692,Частые вопросы и ответы,https://alfabank.ru/sme/payservice/internet-ac...
2,0.848883,1784,PFR_memo.pdf,https://alfabank.servicecdn.ru/media/moscow/re...
3,0.841262,311,От акций до IPO: Расшифровываем язык инвестици...,https://alfabank.ru/help/articles/investments/...
4,0.846137,1705,dogovor_cbo_1072025.pdf,https://alfabank.servicecdn.ru/site-upload/f4/...
5,0.846137,1704,dogovor_cbo_1082025.pdf,https://alfabank.servicecdn.ru/site-upload/e3/...
6,0.840662,1760,ct-12-5062025.pdf,https://alfabank.servicecdn.ru/site-upload/a2/...
7,0.840662,1761,ct-12-30052025.pdf,https://alfabank.servicecdn.ru/site-upload/58/...


{'out': {'status': 'need_clarify',
  'message': 'Похоже на персональный кейс. Уточни продукт/канал/детали (без персональных данных).',
  'results':       score  web_id                                              title  \
  0  0.845569     395  Как заполнить платежное поручение в 2025 году ...   
  1  0.855647     692                            Частые вопросы и ответы   
  2  0.848883    1784                                       PFR_memo.pdf   
  3  0.841262     311  От акций до IPO: Расшифровываем язык инвестици...   
  4  0.846137    1705                            dogovor_cbo_1072025.pdf   
  5  0.846137    1704                            dogovor_cbo_1082025.pdf   
  6  0.840662    1760                                  ct-12-5062025.pdf   
  7  0.840662    1761                                 ct-12-30052025.pdf   
  
                                                   url  \
  0  https://alfabank.ru/help/articles/sme/start/pl...   
  1  https://alfabank.ru/sme/payservice/internet-ac

## Построение разметочного (Gold) датасета

In [38]:
EVAL_N = 200
idx = np.random.RandomState(42).choice(len(q_df), size=EVAL_N, replace=False)
eval_df = q_df.iloc[idx].reset_index(drop=True)
eval_df[["q_id","query"]].head()

,q_id,query
0,5287,Почему не начислен кэшбэк 0% за оплату ЖКУ за ...
1,6019,"Добрый день, прошу прислать надбавку по кредит..."
2,2982,У меня там оставались деньги на счете
3,3348,Что за услуга Альфа чек. И как ее отключить? О...
4,470,У ребёнка не получается войти в лк


In [39]:
from pathlib import Path
import pandas as pd
import re
import shutil

# -----------------------------
# Clarify codes (for need_clarify)
# -----------------------------
CLARIFY_CODES = {
    "1": "how_to_check",
    "2": "how_to_do",
    "3": "ask_product",
    "4": "ask_channel",
    "5": "ask_details",
    "6": "route_support",
    "7": "docs_policy",
    "8": "security_urgent",
}
ALLOWED_STATUS = {"ok", "need_clarify", "no_answer"}

def _pick_clarify_codes():
    print("\nclarify codes:")
    for k, v in CLARIFY_CODES.items():
        print(f"  {k}) {v}")
    s = input("choose codes (e.g. 1,3 or how_to_check,ask_product) or empty: ").strip().lower()
    if not s:
        return ""
    parts = re.split(r"[,\s]+", s)
    codes = []
    for p in parts:
        if p in CLARIFY_CODES:
            codes.append(CLARIFY_CODES[p])
        elif p in set(CLARIFY_CODES.values()):
            codes.append(p)
    return "|".join(sorted(set(codes)))

# -----------------------------
# CSV safety helpers
# -----------------------------
def _backup_csv(path: Path):
    if not path.exists():
        return
    bak = path.with_suffix(path.suffix + ".bak")
    shutil.copy2(path, bak)

def undo_last_label(path="gold_labels.csv"):
    path = Path(path)
    if not path.exists():
        print("No file:", path)
        return
    df = pd.read_csv(path)
    if len(df) == 0:
        print("Empty file.")
        return
    _backup_csv(path)
    last = df.tail(1)
    df = df.iloc[:-1]
    df.to_csv(path, index=False)
    print("Undone last row:")
    display(last)

def edit_label(q_id: int, label_status=None, gold_web_ids=None, clarify_note=None, path="gold_labels.csv"):
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(path)

    df = pd.read_csv(path)
    if "clarify_note" not in df.columns:
        df["clarify_note"] = ""

    mask = df["q_id"].astype(int) == int(q_id)
    if not mask.any():
        print("q_id not found in file:", q_id)
        return

    _backup_csv(path)

    if label_status is not None:
        if label_status not in ALLOWED_STATUS:
            print("bad status:", label_status)
        else:
            df.loc[mask, "label_status"] = label_status

    if gold_web_ids is not None:
        if isinstance(gold_web_ids, (list, tuple)):
            gold_web_ids = ",".join(map(str, gold_web_ids))
        df.loc[mask, "gold_web_ids"] = gold_web_ids

    if clarify_note is not None:
        df.loc[mask, "clarify_note"] = str(clarify_note)

    df.to_csv(path, index=False)
    print("Updated row:")
    display(df[mask])

# -----------------------------
# Rank parsing
# -----------------------------
def _parse_ranks(s: str, n: int):
    s = (s or "").strip()
    if not s:
        return []
    parts = re.split(r"[,\s]+", s)
    ranks = []
    for p in parts:
        p = p.strip()
        if not p:
            continue
        if "-" in p:
            a, b = p.split("-", 1)
            if a.strip().isdigit() and b.strip().isdigit():
                a, b = int(a), int(b)
                lo, hi = min(a, b), max(a, b)
                ranks.extend(range(lo, hi + 1))
        elif p.isdigit():
            ranks.append(int(p))
    return sorted({r for r in ranks if 1 <= r <= n})

# -----------------------------
# Card printing (full previews)
# -----------------------------
def _print_cards(res: pd.DataFrame, ranks=None, max_preview_chars: int = 900):
    """
    ranks: list of 1-based ranks; if None -> print all rows in res
    """
    if res is None or len(res) == 0:
        print("(empty results)")
        return

    if ranks is None:
        idxs = list(range(len(res)))
    else:
        ranks = sorted({int(r) for r in ranks if 1 <= int(r) <= len(res)})
        idxs = [r - 1 for r in ranks]

    print("\n--- PREVIEWS (карточки) ---")
    for j, idx in enumerate(idxs, 1):
        row = res.iloc[idx]
        rank  = row.get("rank", idx + 1)
        web_id = row.get("web_id", "")
        score  = row.get("score", "")
        title  = "" if pd.isna(row.get("title", "")) else str(row.get("title", "")).strip()
        url    = "" if pd.isna(row.get("url", "")) else str(row.get("url", "")).strip()
        prev   = "" if pd.isna(row.get("preview", "")) else str(row.get("preview", "")).strip()
        prev = re.sub(r"\s+", " ", prev)

        if max_preview_chars and len(prev) > max_preview_chars:
            prev = prev[:max_preview_chars] + "..."

        print(f"{j:02d}. rank:{rank} | web_id:{web_id} | score:{score}")
        print(f"    title: {title}")
        print(f"    url:   {url}")
        print(f"    prv:   {prev}\n")

# -----------------------------
# Main labeling loop
# -----------------------------
def label_eval_set(
    eval_df,
    out_csv="gold_labels.csv",
    k_chunks=200,
    k_docs_label=20,
    show_cards=True,          # печатать карточки для всего top-k сразу
    cards_preview_chars=900,  # длина preview в карточках
):
    out_path = Path(out_csv)

    if out_path.exists():
        done = pd.read_csv(out_path)
        # ensure clarify_note column exists
        if "clarify_note" not in done.columns:
            done["clarify_note"] = ""
            _backup_csv(out_path)
            done.to_csv(out_path, index=False)

        done_ids = set(done["q_id"].astype(int))
        print(f"resume: already labeled {len(done_ids)} queries")
    else:
        done_ids = set()
        print("start: no existing labels")

    i = 0
    while i < len(eval_df):
        row = eval_df.iloc[i]
        q_id = int(row["q_id"])
        q = row["query"]

        if q_id in done_ids:
            i += 1
            continue

        print("\n" + "=" * 110)
        print(f"[{i+1}/{len(eval_df)}] q_id={q_id} | {q}")

        resp = safe_search_docs(q, k_chunks=k_chunks, k_docs=k_docs_label)
        print("model:", resp["status"], "—", resp["message"])

        res = resp["results"].copy()
        if len(res):
            res = res.reset_index(drop=True)
            res["rank"] = res.index + 1

            view = res[["rank", "score", "web_id", "title", "url", "preview"]].head(k_docs_label)
            display(view)

            if show_cards:
                _print_cards(view, ranks=None, max_preview_chars=cards_preview_chars)
        else:
            print("(empty results)")

        while True:
            cmd = input(
                "label status [ok/need_clarify/no_answer] "
                "(enter=model, u=undo, e=edit, s=skip, q=quit): "
            ).strip().lower()

            if cmd in ("q", "quit"):
                print("Stopped. Saved:", out_path)
                return

            if cmd in ("s", "skip"):
                print("Skipped q_id", q_id)
                i += 1
                break

            if cmd in ("u", "undo"):
                undo_last_label(out_path)
                if out_path.exists():
                    done = pd.read_csv(out_path)
                    done_ids = set(done["q_id"].astype(int))
                continue

            if cmd.startswith("e"):
                parts = cmd.split()
                if len(parts) == 2 and parts[1].isdigit():
                    eid = int(parts[1])
                else:
                    s_id = input("edit q_id: ").strip()
                    if not s_id.isdigit():
                        print("bad q_id")
                        continue
                    eid = int(s_id)

                new_status = input("new status [ok/need_clarify/no_answer] (empty=keep): ").strip().lower()
                new_status = new_status if new_status in ALLOWED_STATUS else None

                new_gold = input("new gold_web_ids (e.g. 1552,1902) (empty=keep): ").strip()
                new_gold = new_gold if new_gold != "" else None

                new_note = input("new clarify_note (e.g. route_support|ask_product::text) (empty=keep): ").strip()
                new_note = new_note if new_note != "" else None

                edit_label(eid, label_status=new_status, gold_web_ids=new_gold, clarify_note=new_note, path=out_path)

                done = pd.read_csv(out_path)
                done_ids = set(done["q_id"].astype(int))
                continue

            label_status = cmd if cmd else resp["status"]
            if label_status not in ALLOWED_STATUS:
                print("bad status, try again")
                continue

            # ---- choose sources (ok + need_clarify) ----
            gold_ids = []
            if label_status in ("ok", "need_clarify") and len(res):
                if label_status == "need_clarify":
                    print("need_clarify: выбери источники для next-step (как проверить/подключить/условия/куда обратиться).")

                while True:
                    s = input("relevant ranks (e.g. 1,3,5 or 1-3) or empty: ").strip()
                    ranks = _parse_ranks(s, n=min(len(res), k_docs_label))
                    # ranks выбираем только из показанного top-k (head(k_docs_label))
                    gold_ids = [int(res.loc[r - 1, "web_id"]) for r in ranks]
                    gold_ids = sorted(set(gold_ids))

                    print("Selected web_id:", gold_ids)
                    if ranks:
                        # печать карточек выбранных (для подтверждения)
                        _print_cards(res.head(k_docs_label), ranks=ranks, max_preview_chars=cards_preview_chars)
                    else:
                        print("(empty selection)")

                    conf = input("confirm? [y/n] ").strip().lower()
                    if conf in ("y", "yes", ""):
                        break

            # ---- clarify_note (only need_clarify) ----
            clarify_note = ""
            if label_status == "need_clarify":
                codes = _pick_clarify_codes()  # route_support|ask_product ...
                free = input("clarify_note_free (optional short text) or empty: ").strip()
                clarify_note = codes
                if free:
                    clarify_note = f"{codes}::{free}" if codes else f"::{free}"

            out_row = {
                "q_id": q_id,
                "query": q,
                "label_status": label_status,
                "gold_web_ids": ",".join(map(str, gold_ids)),
                "clarify_note": clarify_note,
            }

            first_write = not out_path.exists()
            pd.DataFrame([out_row]).to_csv(out_path, mode="a", header=first_write, index=False)

            done_ids.add(q_id)
            i += 1
            break

    print(f"\nDONE. saved to {out_path}")


In [ ]:
label_eval_set(
    eval_df,
    out_csv="gold_labels.csv",
    k_chunks=200,
    k_docs_label=20,
)


resume: already labeled 150 queries

[151/200] q_id=2190 | Карту закрыла. Почему ее видно в приложении?
model: need_clarify — Похоже на персональный кейс. Уточни продукт/канал/детали (без персональных данных).


,rank,score,web_id,title,url,preview
0,1,0.839484,1760,ct-12-5062025.pdf,https://alfabank.servicecdn.ru/site-upload/a2/...,". 14.4.8. В случаях обнаружения фактов утраты,..."
1,2,0.832912,1761,ct-12-30052025.pdf,https://alfabank.servicecdn.ru/site-upload/58/...,". 14.4.8. В случаях обнаружения фактов утраты,..."
2,3,0.833733,362,"Картотека на расчётном счёте: что это такое, в...",https://alfabank.ru/help/articles/sme/rko/chto...,". То же касается и ситуации, когда клиент приз..."
3,4,0.847371,1030,Как правильно закрыть кредитную карту — грамот...,https://alfabank.ru/help/articles/credit-cards...,. Поэтому лучше закрыть карточку правильно. То...
4,5,0.837453,1522,Что можно делать в приложении?,https://alfabank.ru/help/t/corp/debitcards/deb...,. В приложении можно добавить номер автомобиля...
5,6,0.833070,321,Привязать карту к самозанятости: инструкция от...,https://alfabank.ru/help/articles/selfemployed...,. Для этого есть два способа. Откройте приложе...
6,7,0.846583,1899,Как перевыпустить или заблокировать карту?,https://alfabank.ru/help/t/retail/debitcards/v...,Как перевыпустить или заблокировать карту? Как...
7,8,0.833821,1543,Почему не вижу выплат?,https://alfabank.ru/help/t/corp/debitcards/deb...,Почему не вижу выплат? Общая информация Мобиль...
8,9,0.865822,1588,Как закрыть карту?,https://alfabank.ru/help/t/retail/creditcards/...,Как закрыть карту? Начало использования Общая ...
9,10,0.835063,901,Загрузить справки в систему «Альфа-Банк»,https://alfabank.ru/everyday/payments-and-tran...,Загрузить справки в систему «Альфа-Банк» Получ...



--- PREVIEWS (карточки) ---
01. rank:1 | web_id:1760 | score:0.8394837379455566
    title: ct-12-5062025.pdf
    url:   https://alfabank.servicecdn.ru/site-upload/a2/83/2365/ct-12-5062025.pdf
    prv:   . 14.4.8. В случаях обнаружения фактов утраты, хищения или неправомерного использования Карты и/или CVC2/CVV/CVP2, и/или Средств доступа, и/или Мобильного устройства, использующегося для получения услуги «Альфа-Мобайл», Интернет Банка «Альфа-Клик», Платформы «...

02. rank:2 | web_id:1761 | score:0.8329117894172668
    title: ct-12-30052025.pdf
    url:   https://alfabank.servicecdn.ru/site-upload/58/30/2365/ct-12-30052025.pdf
    prv:   . 14.4.8. В случаях обнаружения фактов утраты, хищения или неправомерного использования Карты и/или CVC2/CVV/CVP2, и/или Средств доступа, и/или Мобильного устройства, использующегося для получения услуги «Альфа-Мобайл», Интернет Банка «Альфа-Клик», Платформы «...

03. rank:3 | web_id:362 | score:0.8337334990501404
    title: Картотека на расчётном счёт

In [ ]:
import pandas as pd
pd.read_csv("gold_labels.csv").tail(50)

In [ ]:
from pathlib import Path
import re
import pandas as pd
import numpy as np

GOLD_PATH = Path("gold_labels.csv")
gold = pd.read_csv(GOLD_PATH)

def parse_ids(x):
    if pd.isna(x): 
        return []
    s = str(x).strip()
    if not s or s.lower() == "nan":
        return []
    return [int(t) for t in re.split(r"[,\s]+", s) if t.strip().isdigit()]

gold["gold_ids"] = gold["gold_web_ids"].apply(parse_ids)

runs = []
for _, r in gold.iterrows():
    query = str(r["query"])
    out = safe_search_docs(query, k_docs=20)   # top-20 для retrieval-метрик

    # ✅ берём предсказанные web_id из out["results"] (DataFrame)
    res = out.get("results")
    pred_ids = []
    top_score = 0.0
    if res is not None and len(res):
        res = res.copy()
        if "score" in res.columns:
            res["score"] = pd.to_numeric(res["score"], errors="coerce").fillna(0.0)
        res = res.sort_values("score", ascending=False)
        pred_ids = [int(x) for x in res["web_id"].head(20).dropna().tolist() if str(x).isdigit()]
        top_score = float(res["score"].head(1).iloc[0]) if "score" in res.columns else 0.0

    meta = out.get("meta") or {}
    # если meta пустая — можно хотя бы top_score/intent добавить
    meta.setdefault("top_score", top_score)
    meta.setdefault("intent", classify_intent(query))

    runs.append({
        "q_id": int(r["q_id"]),
        "query": query,
        "gold_status": r["label_status"],
        "pred_status": out.get("status"),
        "gold_ids": r["gold_ids"],
        "pred_ids": pred_ids,
        "message": out.get("message"),
        **{f"meta_{k}": v for k, v in meta.items()},
    })

runs_df = pd.DataFrame(runs)
runs_df.to_parquet("gold_runs.parquet", index=False)
runs_df.head()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(
    runs_df["gold_status"], runs_df["pred_status"],
    labels=["ok","need_clarify","no_answer"], digits=3
))
print(confusion_matrix(
    runs_df["gold_status"], runs_df["pred_status"],
    labels=["ok","need_clarify","no_answer"]
))

def recall_at_k(gold_ids, pred_ids, k):
    if not gold_ids:
        return np.nan
    return float(any(x in pred_ids[:k] for x in gold_ids))

def mrr_at_k(gold_ids, pred_ids, k):
    if not gold_ids:
        return np.nan
    for i, pid in enumerate(pred_ids[:k], start=1):
        if pid in gold_ids:
            return 1.0 / i
    return 0.0

ok_df = runs_df[runs_df["gold_status"] == "ok"].copy()

for k in [1,3,5,10,20]:
    ok_df[f"recall@{k}"] = ok_df.apply(lambda x: recall_at_k(x["gold_ids"], x["pred_ids"], k), axis=1)
    ok_df[f"mrr@{k}"] = ok_df.apply(lambda x: mrr_at_k(x["gold_ids"], x["pred_ids"], k), axis=1)

ok_df[[c for c in ok_df.columns if c.startswith("recall@") or c.startswith("mrr@")]].mean(numeric_only=True)


In [ ]:
errors = runs_df[runs_df["gold_status"] != runs_df["pred_status"]].copy()
errors.to_csv("gold_errors.csv", index=False)

miss_retrieval = ok_df[ok_df["recall@20"] == 0.0].copy()
miss_retrieval.to_csv("gold_miss_retrieval.csv", index=False)

(len(errors), len(miss_retrieval))


In [ ]:
# разбор ошибок именно по need_clarify
bad_nc = runs_df[(runs_df.gold_status=="need_clarify") & (runs_df.pred_status!="need_clarify")].copy()
print("need_clarify misclassified:", len(bad_nc), "of", (runs_df.gold_status=="need_clarify").sum())

print("\nкуда утекают need_clarify:")
print(bad_nc.pred_status.value_counts())

cols = [c for c in runs_df.columns if c.startswith("meta_")]
show = ["q_id","query","gold_status","pred_status","message"] + cols
display(bad_nc[show].head(30))


## Appendix: Gold labeling UI

## Appendix: competition submission

In [ ]:
# out_rows = []
# for row in tqdm(q_df.itertuples(index=False), total=len(q_df)):
#     preds = top_web_ids(row.query, top_web=5, oversample_chunks=40)
#     out_rows.append({
#         "q_id": int(row.q_id),
#         "web_list": json.dumps(preds, ensure_ascii=False)
#     })

# submit_df = pd.DataFrame(out_rows)

# # Подгоним колонки под пример (на случай если там порядок/имена важны)
# # обычно sample_submission содержит q_id и web_list
# submit_df = submit_df[sub_df.columns.tolist()] if set(sub_df.columns) == set(submit_df.columns) else submit_df

# save_path = os.path.join(PROJECT_DIR, "submit.csv")
# submit_df.to_csv(save_path, index=False)
# print("saved:", save_path)
# display(submit_df.head())